In [1]:
from __future__ import unicode_literals
from textacy import fileio, preprocess, extract, Corpus

In [2]:
summary_docs = fileio.read_file_lines('/home/SMRT-labeled/summary-docs')
summary_meta = fileio.read_json_lines('/home/SMRT-labeled/summary-meta-relabel')

In [3]:
text = [t for t in summary_docs]
meta = [m for m in summary_meta][0]

corpus_train = Corpus('en', texts = text, metadatas = meta)

In [6]:
from collections import Counter

label_list = map(lambda x:x[u'Topic'], meta)

Counter(label_list)

Counter({u'commuter behaviour': 16,
         u'corporate': 34,
         u'corporate social responsibility': 52,
         u'crisis': 64,
         u'customer service': 21,
         u'delay/disruption': 234,
         u'facilities': 288,
         u'fares': 13,
         u'financials': 110,
         u'fines': 3,
         u'general / others': 153,
         u'labour & union': 132,
         u'operations': 11,
         u'rail & engineering': 264,
         u'regulations': 37,
         u'repair/maintenance': 25,
         u'safety/accident': 155,
         u'service announcements': 40,
         u'service excellence': 37,
         u'unknown': 77})

In [4]:
for k in meta[0]:
    print k, meta[0][k]

Category smrt corporate
Topic corporate
Tonality neutral


In [5]:
#topic model
from textacy import vsm

terms_lists = (doc.to_terms_list(ngrams={2, 3}, named_entities=True, as_strings=True) for doc in corpus_train)
doc_term_matrix, id2term = vsm.doc_term_matrix(terms_lists, weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.80, max_n_terms=15000)

In [6]:
#load lable data
label_key = u'Topic'

label_list = map(lambda x:x[label_key], meta)
unique_label = list(set(label_list))
label2id = dict(zip(unique_label, range(len(unique_label))))
label_id = map(lambda x:label2id[x], label_list)

In [7]:
doc_term_matrix

<1766x10607 sparse matrix of type '<type 'numpy.float64'>'
	with 46405 stored elements in Compressed Sparse Row format>

In [8]:
#MLPClassifier requires sklearn >=0.18
#classifier
from sklearn.neural_network import MLPClassifier
#regression
#from sklearn.neural_network import MLPRegressor
#use sknn instead
#from sknn.mlp import Classifier, Layer
import numpy as np

def id2array(id, a_len):
    result = np.zeros(a_len)
    result[id] = 1
    return result

labeltype = len(label2id.keys())

msg_x = doc_term_matrix.toarray()
msg_y = np.array([id2array(x, labeltype) for x in label_id])

In [9]:
clf = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, batch_size=20, hidden_layer_sizes=(500, 100), random_state=1)
clf.fit(msg_x, msg_y)

clf.score(msg_x, msg_y)

0.96884443572124923

In [11]:
#save model
from sklearn.externals import joblib

savefiles = joblib.dump(clf, '/home/pretrain-model/topic-ann.pkl') 

#save meta
fileio.write_json(label2id, '/home/pretrain-model/topic-label2id-ann')
fileio.write_json(id2term, '/home/pretrain-model/topic-id2term-ann')